<a href="https://colab.research.google.com/github/ashishtrivedi16/ML/blob/master/iris_dataset_tensorflow_linear_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split
print("Setup Complete...")

Setup Complete...


In [100]:
col_names = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
iris_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", names = col_names, index_col = False)
print("Dataset loaded...")

Dataset loaded...


In [101]:
iris_data.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [54]:
iris_data.shape

(150, 5)

In [0]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
iris_data['class'] = le.fit_transform(iris_data['class'])

In [0]:
X = iris_data.drop(['class'], axis = 1)
Y = iris_data['class']
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 7)

In [57]:
features = {
        'sepal_length' : x_train['sepal_length'].values,
        'sepal_width' : x_train['sepal_width'].values,
        'petal_length' : x_train['petal_length'].values,
        'petal_width' : x_train['petal_width'].values
    }

feature_column = []
for key in features.keys():
    feature_column.append(tf.feature_column.numeric_column(key = key))
    
print(feature_column)


[_NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [0]:
batch_size = 20
def train_input_fn(x_train, y_train, batch_size):
    
    features = {
        'sepal_length' : x_train['sepal_length'].values,
        'sepal_width' : x_train['sepal_width'].values,
        'petal_length' : x_train['petal_length'].values,
        'petal_width' : x_train['petal_width'].values
    }
    
    labels = y_train.values.astype(int)
    
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(50).repeat().batch(10)
    
    return dataset

In [0]:
def eval_input_fn(x_test, y_test, batch_size):

    features = {
        'sepal_length' : np.array(x_test['sepal_length']),
        'sepal_width' : np.array(x_test['sepal_width']),
        'petal_length' : np.array(x_test['petal_length']),
        'petal_width' : np.array(x_test['petal_width'])
    }
    
    if y_test is None:
        inputs = features
    else:
        labels = y_test.values.astype(int)
        inputs = (features, labels)
        
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    dataset = dataset.batch(batch_size)
    
    return dataset

In [114]:
estimator = tf.estimator.LinearClassifier(feature_columns = feature_column, n_classes = 3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmps5iu8sod', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe6e0d5c908>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [118]:
estimator.train(input_fn = lambda : train_input_fn(x_train, y_train, batch_size), steps = 1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmps5iu8sod/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 10000 into /tmp/tmps5iu8sod/model.ckpt.
INFO:tensorflow:loss = 0.52523714, step = 10001
INFO:tensorflow:global_step/sec: 758.443
INFO:tensorflow:loss = 0.17519696, step = 10101 (0.138 sec)
INFO:tensorflow:global_step/sec: 1116.79
INFO:tensorflow:loss = 0.69081813, step = 10201 (0.085 sec)
INFO:tensorflow:global_step/sec: 1143.57
INFO:tensorflow:loss = 0.4585027, step = 10301 (0.091 sec)
INFO:tensorflow:global_step/sec: 1158.39
INFO:tensorflow:loss = 1.8817445, step = 10401 (0.085 sec)
INFO:tensorflow:global_step/sec: 1164.08
INFO:tensorflow:loss = 0.8961264, step = 10501 (0.084 sec)
INFO:tensorflow:global_step/sec: 1192.51
INFO:tensorflow:l

In [119]:
eval_result = estimator.evaluate(input_fn = lambda : eval_input_fn(x_test, y_test, batch_size))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-11-06:48:24
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmps5iu8sod/model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-11-06:48:24
INFO:tensorflow:Saving dict for global step 11000: accuracy = 0.93333334, average_loss = 0.163339, global_step = 11000, loss = 2.4500852
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 11000: /tmp/tmps5iu8sod/model.ckpt-11000

Test set accuracy: 0.933



In [120]:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'sepal_length': [5.1, 5.9, 6.9],
    'sepal_width': [3.3, 3.0, 3.1],
    'petal_length': [1.7, 4.2, 5.4],
    'petal_width': [0.5, 1.5, 2.1],
}

predictions = estimator.predict(input_fn = lambda : eval_input_fn(predict_x, 
                                                                  None,
                                            batch_size = batch_size))


template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(SPECIES[class_id],
                          100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmps5iu8sod/model.ckpt-11000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (99.1%), expected "Setosa"

Prediction is "Versicolor" (91.4%), expected "Versicolor"

Prediction is "Virginica" (94.3%), expected "Virginica"
